In [1]:
import os

from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

from langchain_openai import ChatOpenAI
llm_model = 'gpt-4o-mini'
chat_model = ChatOpenAI(temperature=0.7, model=llm_model)

### Simple Example for Embeddings

In [2]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
embeddings_2 = OpenAIEmbeddings(model="text-embedding-3-small")
# Embeddings - simpler example - compare similarity etc.
text1 = "I love a cat."
text2 = "This is a rock."
text3 = "I like the dog."

embed1 = embeddings.embed_query(text1)
embed12 = embeddings_2.embed_query(text1)
embed2 = embeddings.embed_query(text2)
embed3 = embeddings.embed_query(text3)
print(f"Embed! == {len(embed1)}")
import numpy as np
similarity_13 = np.dot(embed1, embed3)
similarity_12 = np.dot(embed1, embed2)
print(f"Similary of text1 and text2: {similarity_12*100}%")
print(f"Similary of text1 and text3: {similarity_13*100}%")


print(embed1[:3], embed12[:3])

Embed! == 512
Similary of text1 and text2: 19.71148677625692%
Similary of text1 and text3: 48.524775267896196%
[0.00929352268576622, -0.0662163496017456, -0.08923234045505524] [0.006161571945995092, -0.043901197612285614, -0.05916071683168411]


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([embed1], [embed2]))

[[0.19711487]]


### Split document into chunks

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("./data/react-paper.pdf")
pages = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

texts = text_splitter.split_documents(pages)
print("쪼개진 Chunk의 개수: ", len(texts))

쪼개진 Chunk의 개수:  33


In [10]:
from langchain_chroma import Chroma
persist_directory = "./data/db/chroma"

vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings, # openai embeddings
    persist_directory=persist_directory
    )
print(vectorstore._collection.count())

33


In [13]:
query = "what do they say about ReAct prompting method?"
print(vectorstore.similarity_search(query))

[Document(metadata={'page': 4, 'source': './data/react-paper.pdf'}, page_content='Published as a conference paper at ICLR 2023\nPrompt Methoda HotpotQA Fever\n(EM) (Acc)\nStandard 28.7 57.1\nCoT(Wei et al., 2022) 29.4 56.3\nCoT-SC(Wang et al., 2022a) 33.4 60.4\nAct 25.7 58.9\nReAct 27.4 60.9\nCoT-SC→ReAct 34.2 64.6\nReAct→CoT-SC 35.1 62.0\nSupervised SoTAb 67.5 89.5\nTable 1: PaLM-540B prompting results on\nHotpotQA and Fever.\naHotpotQA EM is 27.1, 28.9, 33.8 for Standard, CoT,\nCoT-SC in Wang et al. (2022b).\nb(Zhu et al., 2021; Lewis et al., 2020)\n0 5 10 15 20\n#CoT-SC trials\n26\n28\n30\n32\n34HotpotQA EM\n0 5 10 15 20\n#CoT-SC trials\n47.5\n50.0\n52.5\n55.0\n57.5\n60.0\n62.5\n65.0Fever Acc\nMethod\nCoT-SC -> ReAct\nReAct -> CoT-SC\nCoT-SC\nReAct\nCoT\nFigure 2: PaLM-540B prompting results with respect to\nnumber of CoT-SC samples used.\nsearch reformulation (“maybe I can search/look up x instead”), and synthesize the ﬁnal answer (“...so\nthe answer is x”). See Appendix C for more

### LangChain Retrievers

In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("Tell me more about ReAct prompting")
# print(retriever.search_type)
print(docs[0].page_content)

/var/folders/d2/b7bnbh8d1mx5rjyb450xlt200000gn/T/ipykernel_36436/1833132863.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Tell me more about ReAct prompting")


Published as a conference paper at ICLR 2023
Method Pick Clean Heat Cool Look Pick 2 All
Act(best of 6) 88 42 74 67 72 41 45
ReAct(avg) 65 39 83 76 55 24 57
ReAct(best of 6) 92 58 96 86 78 41 71
ReAct-IM(avg) 55 59 60 55 23 24 48
ReAct-IM(best of 6) 62 68 87 57 39 33 53
BUTLERg (best of 8) 33 26 70 76 17 12 22
BUTLER(best of 8) 46 39 74 100 22 24 37
Table 3: AlfWorld task-speciﬁc success rates (%). BUTLER and
BUTLERg results are from Table 4 of Shridhar et al. (2020b). All
methods use greedy decoding, except that BUTLER uses beam search.
Method Score SR
Act 62.3 30.1
ReAct 66.6 40.0
IL 59.9 29.1
IL+RL 62.4 28.7
Human 82.1 59.6Expert
Table 4: Score and suc-
cess rate (SR) on Web-
shop. IL/IL+RL taken
from Yao et al. (2022).
trained with 1,012 human annotated trajectories, and a imitation + reinforcement learning (IL + RL)
method additionally trained with 10,587 training instructions.
Results ReAct outperforms Act on both ALFWorld (Table 3) and Webshop (Table 4). On
ALFWorld, the bestReA

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
        You are an assistant for question-answering tasks. 
        Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. 
        Use three sentences maximum and keep the answer concise.

        Question: {question} 
        Context: {context} 
    """
)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vectorstore.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | chat_model
    | StrOutputParser()
)

qa_chain.invoke(query)

'ReAct prompting, presented in a 2023 ICLR paper, integrates reasoning and action within large language models (LLMs) to enhance decision-making in interactive environments. It significantly outperforms traditional methods like Act and IM by allowing the model to generate thoughts alongside actions, fostering better goal decomposition and commonsense reasoning. This approach has shown improved performance in tasks such as ALFWorld and WebShop, demonstrating its effectiveness in complex, knowledge-intensive scenarios.'

In [23]:
# rm -rf ./data/db/chroma
# Chroma 삭제